In [1]:
# Here, we are going to read in a pcap file and parse it using scapy

import os
import sys
import time
import datetime
import socket
import pathlib
import numpy as np
import pandas as pd
# import struct
# import binascii
# import dpkt
# import scapy
import scapy.all as scapy
from collections import Counter

In [2]:
!pip install scapy


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install numpy

In [ ]:
!pip install pandas

In [2]:
args: list = [
    './first.pcap',
    100,
    'DDoS'
]

In [3]:
filename = args[0]
num_packets = args[1]
label = args[2]

In [4]:
packets = scapy.rdpcap(filename)

In [5]:
for p in packets:
    packet = (p/scapy.Ether())
    break

In [6]:
# print(packet)
k = packet.show()

###[ Ethernet ]### 
  dst       = 90:b1:1c:a2:c0:d3
  src       = 70:f3:5a:42:73:e8
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 544
     id        = 2
     flags     = DF
     frag      = 0
     ttl       = 50
     proto     = udp
     chksum    = 0xa80c
     src       = 172.16.0.5
     dst       = 192.168.50.1
     \options   \
###[ UDP ]### 
        sport     = 42217
        dport     = 20168
        len       = 524
        chksum    = 0x2852
###[ Raw ]### 
           load      = '\x00\x03\x00\x01\x7fELF\x01\x02\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x08\x00\x00\x00\x01\x00@V \x00\x00\x004\x00\x05b\x1cP\x00\x10\x07\x004\x00 \x00\x06\x00(\x00\x1b\x00\x1a\x00\x00\x00\x06\x00\x00\x004\x00@\x004\x00@\x004\x00\x00\x00\\xc0\x00\x00\x00\\xc0\x00\x00\x00\x05\x00\x00\x00\x04\x00\x00\x00\x03\x00\x00\x00\\xf4\x00@\x00\\xf4\x00@\x00\\xf4\x00\x00\x00\x14\x00\x00\x00\x14\x00\x00\x00\x04\x00\x00\x00\x01p\x00\x00\x00\x00\x

In [8]:
#make a generator that processes num_packets at a time

def getSample():
    i: int = 0
    current_sample: list = []

    for p in packets:

        if  i > 0 and i % num_packets == 0:
            i+=1
            yield current_sample
        else:
            packet = p/scapy.Ether()
            current_sample.append(packet)
            i+=1

In [ ]:
sample[0]

NameError: name 'sample' is not defined

In [16]:
samples = getSample()
sample = next(samples)
sample_values = {}

temp = {}

temp['IP_len'] = []
temp['destination_len'] = []
temp['payload_len'] = []
temp['UDP_len'] = []
temp['udp_checksum'] = []

# Additional features
temp['dst_mac'] = []  # Destination MAC addresses
temp['src_mac'] = []  # Source MAC addresses
temp['eth_type'] = []  # Ethernet types
temp['ip_version'] = []  # IP versions
temp['ip_ihl'] = []  # IP header lengths
temp['ip_tos'] = []  # IP Type of Service values
temp['ip_flags'] = []  # IP fragmentation flags
temp['ip_ttl'] = []  # IP Time to Live values
temp['ip_proto'] = []  # IP protocols (e.g., UDP, TCP, ICMP, etc.)
temp['src_ip'] = []  # Source IP addresses
temp['udp_sport'] = []  # UDP source ports
temp['udp_dport'] = []  # UDP destination ports

first_pkt_time = sample[0].time  # Get the timestamp of the first packet
last_pkt_time = sample[-1].time  # Get the timestamp of the last packet
duration = last_pkt_time - first_pkt_time  # Compute the duration of the capture in second

for p in sample:

    temp['IP_len'].append(p.len)
    
    # Extracting Ethernet layer information
    if p.haslayer('Ethernet'):
        temp['dst_mac'].append(p['Ethernet'].dst)
        temp['src_mac'].append(p['Ethernet'].src)
        temp['eth_type'].append(p['Ethernet'].type)
    
    # Extracting IP layer information
    if p.haslayer('IP'):
        temp['ip_version'].append(p['IP'].version)
        temp['ip_ihl'].append(p['IP'].ihl)
        temp['ip_tos'].append(p['IP'].tos)
        temp['ip_flags'].append(p['IP'].flags)
        temp['ip_ttl'].append(p['IP'].ttl)
        temp['ip_proto'].append(p['IP'].proto)
        temp['src_ip'].append(p['IP'].src)
        
        destination_len = len(p['IP'].dst)
        temp['destination_len'].append(destination_len)

    # Extracting Raw layer information   
    if p.haslayer('Raw'):
        temp['payload_len'].append(len(p['Raw'].load))
    
    # Extracting UDP layer information
    if p.haslayer('UDP'):
       udp_pkt = p['UDP']
       udp_len = len(udp_pkt.payload)
       temp['UDP_len'].append(udp_len)
       
       udp_checksum = p['UDP'].chksum
       temp['udp_checksum'].append(udp_checksum)
       
       temp['udp_sport'].append(p['UDP'].sport)# UDP source port
       temp['udp_dport'].append(p['UDP'].dport)  # UDP destination port


#length of the ip packet
sample_values['IP_len_avg'] = sum(temp['IP_len'])/len(temp['IP_len'])
sample_values['IP_len_max'] = max(temp['IP_len'])
sample_values['IP_len_min'] = min(temp['IP_len'])
sample_values['IP_len_std'] = np.std(temp['IP_len'])
sample_values['IP_len_var'] = np.var(temp['IP_len'])
sample_values['IP_len_sum'] = sum(temp['IP_len'])

#length of the destination IP address in bytes
sample_values['destination_len_avg'] = sum(temp['destination_len']) / len(temp['destination_len'])
sample_values['destination_len_max'] = max(temp['destination_len'])
sample_values['destination_len_min'] = min(temp['destination_len'])
sample_values['destination_len_std'] = np.std(temp['destination_len'])
sample_values['destination_len_var'] = np.var(temp['destination_len'])
sample_values['destination_len_sum'] = sum(temp['destination_len'])

#length of the payload (or data) of each packet.
sample_values['payload_len_avg'] = sum(temp['payload_len'])/len(temp['payload_len'])
sample_values['payload_len_max'] = max(temp['payload_len'])
sample_values['payload_len_min'] = min(temp['payload_len'])
sample_values['payload_len_std'] = np.std(temp['payload_len'])
sample_values['payload_len_var'] = np.var(temp['payload_len'])
sample_values['payload_len_sum'] = sum(temp['payload_len'])

#length of the UDP (User Datagram Protocol) payload in each packet
sample_values['UDP_len_avg'] = sum(temp['UDP_len'])/len(temp['UDP_len'])
sample_values['UDP_len_max'] = max(temp['UDP_len'])
sample_values['UDP_len_min'] = min(temp['UDP_len'])
sample_values['UDP_len_std'] = np.std(temp['UDP_len'])
sample_values['UDP_len_var'] = np.var(temp['UDP_len'])
sample_values['UDP_len_sum'] = sum(temp['UDP_len'])

sample_values['udp_checksum_avg'] = sum(temp['udp_checksum'])/len(temp['udp_checksum'])
sample_values['udp_checksum_max'] = max(temp['udp_checksum'])
sample_values['udp_checksum_min'] = min(temp['udp_checksum'])
sample_values['udp_checksum_std'] = np.std(temp['udp_checksum'])
sample_values['udp_checksum_var'] = np.var(temp['udp_checksum'])
sample_values['udp_checksum_dist'] = Counter(temp['udp_checksum'])

sample_values['dst_mac_dist'] = Counter(temp['dst_mac'])
sample_values['src_mac_dist'] = Counter(temp['src_mac'])
sample_values['eth_type_dist'] = Counter(temp['eth_type'])
sample_values['ip_version_dist'] = Counter(temp['ip_version'])
sample_values['ip_ihl_dist'] = Counter(temp['ip_ihl'])
sample_values['ip_tos_dist'] = Counter(temp['ip_tos'])
sample_values['ip_flags_dist'] = Counter(temp['ip_flags'])
sample_values['ip_ttl_dist'] = Counter(temp['ip_ttl'])
sample_values['ip_proto_dist'] = Counter(temp['ip_proto'])
sample_values['src_ip_dist'] = Counter(temp['src_ip'])
sample_values['udp_sport_dist'] = Counter(temp['udp_sport'])
sample_values['udp_dport_dist'] = Counter(temp['udp_dport'])

sample_values['avg_packet_rate'] = len(sample) / duration 

sample_values['Label'] = label


In [17]:
dataset_raw = [
    sample_values,
]

dataset = pd.DataFrame(dataset_raw)

In [18]:
dataset.head()

,IP_len_avg,IP_len_max,IP_len_min,IP_len_std,IP_len_var,IP_len_sum,destination_len_avg,destination_len_max,destination_len_min,destination_len_std,...,ip_ihl_dist,ip_tos_dist,ip_flags_dist,ip_ttl_dist,ip_proto_dist,src_ip_dist,udp_sport_dist,udp_dport_dist,avg_packet_rate,Label
0,544.0,544,544,0.0,0.0,54400,12.0,12,12,0.0,...,{5: 100},"{0: 98, 32: 2}",{DF: 100},"{50: 14, 51: 13, 55: 12, 47: 12, 49: 13, 45: 2...",{17: 100},{'172.16.0.5': 100},"{42217: 2, 62896: 1, 63624: 2, 62640: 2, 63496...","{20168: 2, 42948: 1, 60749: 2, 49218: 2, 45761...",61425.06142506142506142506143,DDoS
